In [9]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [10]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [11]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text

LOG = rts.utils.get_logger()

In [75]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
sample_df = rts.metadata.get_one_percent_sample(df)
# rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# rts.metadata.export_metadata_stats(df, OUTDIR)
sample_df.head()

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
mediaId,,,,,,,,,,,,,,,,,,,,
ZE003073,AA1101000650,/mnt/rts/3/7/0/ZE003073,2932,16:9,SD,2010-10-10T20:05:00Z,Programme,Programme,Autres (information),"[BETISIER, BIO]",Mise au point,TSR 1,Restriction/Condition,PETITS MEURTRES ENTRE AMIS // UN LIT POUR DEUX...,"* SUISSE : 20101010, l'inspecteur HEIMO enquêt...",[SUISSE],[00:02.36\r\nPlateau : diverses présentations ...,1670505072,1286741100,"{'892779': {'guid': 'AA1101000650', 'mediaId':..."
ZE003216,AA1101000660,/mnt/rts/6/1/2/ZE003216,3043,16:9,SD,2010-11-14T00:00:00Z,Programme,Programme,Autres (information),None,Mise au point,TSR 1,Restriction/Condition,BERLUSCONNERIES ? // SERPENT DU LAC // L'ADIEU...,"* ITALIE : 20101114 et archives, un nouveau sc...","[VAUD, ITALIE, PLATEAU, SUISSE, SAXON, TOUR-DE...",[00:02.38\r\nPlateau : diverses présentation d...,1670505072,1289692800,"{'893397': {'guid': 'AA1101000660', 'mediaId':..."
ZE010504,AA1101000710,/mnt/rts/4/0/5/ZE010504,278,16:9,SD,2010-10-31T00:00:00Z,Programme,Programme,Autres (information),None,Ensemble,TSR 1,Inconnu,PRO JUVENTUTE - 10.10.31,"* SUISSE : 20101031, présentation de la Fondat...",[SUISSE],"[00:01.00\r\nSUISSE, int salle de classe : div...",1670505072,1288483200,"{'894120': {'guid': 'AA1101000710', 'mediaId':..."
ZE003393,AA1101000730,/mnt/rts/3/9/3/ZE003393,1922,16:9,SD,2011-01-10T00:00:00Z,Programme,Programme,Folklore et musique populaire,None,La boîte à musique,TSR 2,Inconnu,4E TROPHEE DES MUSIQUES POPULAIRES : 1ERE DEMI...,"* Moudon, Plateau : 20101218 et archives, émis...","[MOUDON, PLATEAU]","[ Moudon, diverses séquences : diverses présen...",1670505072,1294617600,"{'895543': {'guid': 'AA1101000730', 'mediaId':..."
ZE003206,AA1101000740,/mnt/rts/6/0/2/ZE003206,744,16:9,SD,2010-11-23T00:00:00Z,Programme,Programme,"Jeux, concours",None,Tapis rouge,TSR 1,Restriction/Condition,10.11.23,None,None,None,1670505072,1290470400,{}


In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)

In [ ]:
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [ ]:
# Extract location from transcripts and build dataframe
ts = rts.pipeline.load_all_transcripts(LOCAL_VIDEOS)
fts = rts.features.text.build_location_df(ts)
fts = rts.metadata.merge_location_df_with_metadata(sample_df, fts)
fts.head()

In [7]:
import PIL
import os
from PIL import ImageOps, Image

In [51]:
im = Image.open(os.path.join(OUTDIR, 'test_image.jpg'))
rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

{'original': 'data/test_image_pyramid/original/test_image.jpg',
 '32px': 'data/test_image_pyramid/32px/test_image.jpg',
 '64px': 'data/test_image_pyramid/64px/test_image.jpg',
 '128px': 'data/test_image_pyramid/128px/test_image.jpg',
 '256px': 'data/test_image_pyramid/256px/test_image.jpg',
 '512px': 'data/test_image_pyramid/512px/test_image.jpg'}

In [41]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# rts.io.media.get_media_info(video_path)

scenes = rts.io.media.detect_scenes(video_path)
scenes = rts.io.media.filter_scenes(scenes, 5)
paths = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)

00:00:00.040
00:00:21.200
00:00:42.320
00:00:47.280
00:00:50.000
00:00:52.680
00:00:56.320
00:01:00.360
00:01:04.360
00:01:05.120
00:01:07.720
00:01:10.280
00:01:14.800
00:01:20.840
00:01:26.840
00:01:32.080
00:01:41.840
00:01:51.560
00:01:53.720
00:01:56.320
00:01:58.840
00:02:11.480
00:02:17.560
00:02:23.600
00:02:42.120
00:02:47.400
00:02:52.640
00:02:52.760
00:02:59.560
00:03:06.320
00:03:09.200
00:03:11.920
00:03:14.560
00:03:19.640
00:03:29.640
00:03:39.600
00:03:39.720
00:03:44.160
00:03:48.560
00:03:48.680
00:03:51.200
00:03:53.680
00:04:01.120
00:04:04.920
00:04:08.680
00:04:08.800
00:04:15.960
00:04:23.080
00:04:31.680
00:04:34.160
00:04:36.600
00:04:36.720
00:04:43.760
00:04:50.760
00:04:56.880
00:05:04.320
00:05:11.720
00:05:14.920
00:05:17.720
00:05:20.480
00:05:20.600
00:05:34.640
00:05:48.640
00:06:01.920
00:06:07.720
00:06:13.480
00:06:22.680
00:06:26.280
00:06:29.800
00:06:29.920
00:06:32.560
00:06:35.120
00:06:42.000
00:06:47.280
00:06:52.560
00:06:55.840
00:06:58.360

In [85]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)


rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=True, force_scene=True, force_trans=False)

{'mediaId': 'ZB001000', 'status': 'success', 'error': ''}

In [ ]:
%ls {remux_folder}

In [15]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

,location,s,e,t
media_id,,,,
ZT017300,183,183,183,183
ZB029600,49,49,49,49
ZB051900,49,49,49,49
ZB050100,45,45,45,45
ZB018100,40,40,40,40
...,...,...,...,...
ZB266700,1,1,1,1
ZT042300,1,1,1,1
ZB140200,1,1,1,1


In [16]:
fts

,media_id,location,s,e,t
0,ZB080800,Genève,76.18,78.46,"Tant à Berne qu'à Genève,"
1,ZB080800,Bern,76.18,78.46,"Tant à Berne qu'à Genève,"
2,ZB066800,Genève,156.00,159.00,la voix de Genève et de la Suisse du sud.
3,ZB066800,Genève,284.00,286.00,"ou beaucoup d'organisations basées sur la Genève,"
4,ZB066800,Genève,1276.00,1277.00,de Genève.
...,...,...,...,...,...
2217,ZB004400,Lausanne,1146.76,1148.76,Romain étudie le graphisme à Lausanne
2218,ZB004400,Sitten,1148.96,1150.96,et vit à Sion.
2219,ZB004400,Lausanne,1364.36,1367.36,lors d'un défilé le 1er mars à Lescobar à Laus...
2220,ZB004400,Fribourg,1382.16,1385.16,"Il y aura des chansons, de Châtel Saint-Denis,..."


In [7]:
import weaviate

client = weaviate.Client("http://localhost:8080") # or another location where your Weaviate instance is running

client.schema.get() # get the full schema as example

{'classes': []}

In [34]:
swiss_cities = rts.features.text.get_swiss_cities()

In [36]:
swiss_cities['Genève']

{'lat': 46.20222, 'lon': 6.14569, 'geoid': 2660646}

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

In [ ]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

In [ ]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [ ]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

In [ ]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})